In [ ]:
import panel as pn
pn.extension()

The `ChatFeed` is a low-level widget, that lets you manage a list of [`ChatEntry`](ChatEntry.ipynb) items.

This widget provides backend methods to:
- Send (append) messages to the chat log.
- Stream tokens to the latest `ChatEntry` in the chat log.
- Execute callbacks when a user sends a message.
- Undo a number of sent `ChatEntry` objects.
- Clear the chat log of all `ChatEntry` objects.

See [`ChatInterface`](ChatInterface.ipynb) for a high-level, *easy to use*, *ChatGPT like* interface.

![Chat Design Specification](../../assets/ChatDesignSpecification.png)

#### Parameters:

##### Core

* **`callback`** (callable): Callback to execute when a user sends a message.
* **`value`** (`List[ChatEntry]`): The list of entries in the feed.

##### Styling

* **`card_params`** (Dict): Parameters to pass to Card, such as `header`, `header_background`, `header_color`, etc.
* **`entry_params`** (Dict): Parameters to pass to each ChatEntry, such as `reaction_icons`, `timestamp_format`, `show_avatar`, `show_user`, and `show_timestamp`.

##### Other

* **`auto_scroll_limit`** (int): Max pixel distance from the latest object in the Column to activate automatic scrolling upon update. Setting to 0 disables auto-scrolling.
* **`auto_scroll_limit`** (int): Min pixel distance from the latest object in the Column to display the scroll button. Setting to 0 disables the scroll button.
* **`callback_avatar`** (str | BinaryIO): The avatar to use for the user. Can be a single character text, an emoji, or anything supported by `pn.pane.Image`. If not set, uses the first character of the name.
* **`callback_user`** (str): The default user name to use for the entry provided by the callback.
* **`header`** (string or viewable): The header of the chat feed.
* **`placeholder_text`** (any): If placeholder is the default LoadingSpinner, the text to display next to it.
* **`placeholder_threshold`** (float): Min duration in seconds of buffering before displaying the placeholder. If 0, the placeholder will be disabled. Defaults to 0.2.
* **`placeholder`** (any): Placeholder to display while the callback is running. Defaults to a LoadingSpinner.
* **`view_latest`** (bool): Whether to scroll to the latest object on init. If not enabled the view will be on the first object. Defaults to True.
and the component `instance`.
the previous message value `contents`, the previous `user` name,
when `respond` is called. The signature must include

#### Properties

* **`entries`** (List[ChatEntry]): Gets and sets the entries in the feed.

#### Methods

##### Core

* **`send`**: Sends a value and creates a new entry in the chat log.
* **`stream`**: Streams a token and updates the provided entry,

##### Other

* **`clear`**: Clears the chat log and returns the entries that were cleared.
* **`respond`**: Executes the callback with the latest entry in the chat log.
* **`undo`**: Removes the last `count` of entries from the chat log and returns them. Default `count` is 1.

___

## Introduction

In [ ]:
from asyncio import sleep
from panel.widgets.chat import ChatEntry, ChatFeed

ASSISTANT_AVATAR = "https://upload.wikimedia.org/wikipedia/commons/6/63/Yumi_UBports.png"
SYSTEM_AVATAR = "https://upload.wikimedia.org/wikipedia/commons/6/6d/Windows_Settings_app_icon.png"
USER_AVATAR = "https://upload.wikimedia.org/wikipedia/commons/f/f0/GirlB.png"

You can create a basic chat feed as follows

In [ ]:
chat_feed = pn.widgets.ChatFeed(height=500)
chat_feed

You can send chat entries with the `send` method

In [ ]:
chat_feed.send("Hello world!", user="Bot", avatar="🤖")

Besides messages of `str` type, the `send` method can also accept `dict`s containing the key `value` and `ChatEntry` objects.

In [ ]:
chat_feed.send({"value": "Welcome!", "user": "Bot", "avatar": "B"})

Note if you provide both the user/avatar in the `dict` and keyword argument, the keyword argument takes precedence.

In [ ]:
entry = chat_feed.send({"value": "Overtaken!", "user": "Bot"}, user="MegaBot")

The `send` method returns a [`ChatEntry`](ChatEntry.ipynb).

In [ ]:
entry

The `ChatEntry` can display any object that Panel can display. You can **interact with chat entries** like any other Panel component. You can find examples in the [`ChatEntry` Reference Notebook](ChatEntry.ipynb).

You can see the list of chat entries via the `entries` property. TODO: MAKE THIS POSSIBLE VIA `value` PARAMETER. And enable binding to the `value` parameter too.

In [ ]:
chat_feed.entries

You can create a basic `ChatFeed` with pre-populated chat entries as shown below

In [ ]:
chat_feed = ChatFeed(
    value=[
        ChatEntry(user="System", value="You are a friendly assistant", avatar=SYSTEM_AVATAR),
        ChatEntry(user="Assistant", value="Hi There. How my I help?", avatar=ASSISTANT_AVATAR),
        ],
    height=375
)
chat_feed

## Responding via a `callback` function

You can provide a `callback` function to handle new chat entries and to run when the `respond` method is called.

The `callback` function can be either a *sync* or and *async* function. We recommend using an *async* function whenever possible. This will help your application to stay responsive and handle more users.

Many LLM frameworks provides *easy to use*, *async* functions and methods. See for example [`openai` | Async API](https://github.com/openai/openai-python#async-api).

In [ ]:
async def get_response(contents, user, instance):
    await sleep(1)
    return "I dont' know"

chat_feed = ChatFeed(
    value=[
        ChatEntry(user="System", value="You are a friendly assistant", avatar=SYSTEM_AVATAR, visible=False),
        ChatEntry(user="Assistant", value="Hi There. How my I help?", avatar=ASSISTANT_AVATAR),
        ],
    callback=get_response,
    callback_avatar=ASSISTANT_AVATAR,
    height=400
)
chat_feed

In [ ]:
entry = chat_feed.send(user="User", value="What is 2+2?", avatar=USER_AVATAR)

## Responding via a *generator* `callback` function.

You can provide very advanced responses like *streaming* if your `callback` is a generator function. I.e. one that
uses `yield` to return multiple values.

Lets start with a basic example where the `ChatEntry` value is replaced.

In [ ]:
async def get_response(contents, user, instance):
    for response in ["That is a very good question", "Let me think...", "Sorry, I don't know"]:
        await sleep(1.5)
        yield response
        

chat_feed = ChatFeed(
    value=[
        ChatEntry(user="System", value="You are a friendly assistant", avatar=SYSTEM_AVATAR, visible=False),
        ChatEntry(user="Assistant", value="Hi There. How my I help?", avatar=ASSISTANT_AVATAR),
        ],
    callback=get_response,
    callback_avatar=ASSISTANT_AVATAR,
    height=400
)
chat_feed

In [ ]:
entry = chat_feed.send(user="User", value="What is 2+2?", avatar=USER_AVATAR)

Here is an example where the `ChatEntry` value is extended multiple times.

In [ ]:
async def get_response(contents, user, instance):
    loading_spinner = pn.indicators.LoadingSpinner(value=True, size=25, color="dark")
    layout = pn.Column(loading_spinner)
    entry = ChatEntry(user="Assistant", value=layout, avatar=ASSISTANT_AVATAR)

    for response in ["That is a very good question", "Let me think", "Sorry, I don't know"]:
        await sleep(1.5)
        layout.insert(-1, response)
        yield entry
    
    layout.pop(-1)
    yield entry
        

chat_feed = ChatFeed(
    value=[
        ChatEntry(user="System", value="You are a friendly assistant", avatar=SYSTEM_AVATAR, visible=False),
        ChatEntry(user="Assistant", value="Hi There. How my I help?", avatar=ASSISTANT_AVATAR),
        ],
    callback=get_response,
    callback_avatar=ASSISTANT_AVATAR,
    height=400
)
chat_feed

In [ ]:
entry = chat_feed.send(user="User", value="What is 2+2?", avatar=USER_AVATAR)

Here is an example where tokens are streamed to the `ChatEntry` value

In [ ]:
async def get_response(contents, user, instance):
    entry = ChatEntry(user="Assistant", value="", avatar=ASSISTANT_AVATAR)

    for token in ["That ", "is ", "a ", "very ", "good ", "question. ", "Let ", "me ", "think ", "Sorry, ", "I ", "don't ", "know."]:
        await sleep(0.2)
        entry.value += token
        yield entry
        

chat_feed = ChatFeed(
    value=[
        ChatEntry(user="System", value="You are a friendly assistant", avatar=SYSTEM_AVATAR, visible=False),
        ChatEntry(user="Assistant", value="Hi There. How my I help?", avatar=ASSISTANT_AVATAR),
        ],
    callback=get_response,
    callback_avatar=ASSISTANT_AVATAR,
    height=400
)
chat_feed

In [ ]:
entry = chat_feed.send(user="User", value="What is 2+2?", avatar=USER_AVATAR)

## Custom Chat Interface

In [ ]:
from asyncio import sleep
import panel as pn
from panel.widgets.chat import ChatEntry, ChatFeed

pn.extension()


async def get_response(contents, user, instance):
    await sleep(0.5)
    return {
        "Philipp": None,  # Ignore Philipp :-)
        "Marc": "It is 2",
        "Andrew": "It is 4",
    }.get(user, "I don't know")


ASSISTANT_AVATAR = (
    "https://upload.wikimedia.org/wikipedia/commons/6/63/Yumi_UBports.png"
)

chat_feed = ChatFeed(
    value=[ChatEntry(user="Assistant", value="Hi There!", avatar=ASSISTANT_AVATAR)],
    callback=get_response,
    height=500,
    callback_avatar=ASSISTANT_AVATAR,
)

marc_button = pn.widgets.Button(
    name="Marc",
    on_click=lambda event: chat_feed.send(
        user="Marc", value="What is the square root of 4?", avatar="🚴"
    ),
    align="center",
    disabled=chat_feed.param.disabled
)
andrew_button = pn.widgets.Button(
    name="Andrew",
    on_click=lambda event: chat_feed.send(
        user="Andrew", value="What is the square root of 4 squared?", avatar="🏊"
    ),
    align="center",
    disabled=chat_feed.param.disabled
)
philipp_button = pn.widgets.Button(
    name="Philipp",
    on_click=lambda event: chat_feed.send(
        user="Philipp", value="What is 2+2", avatar="🏃"
    ),
    align="center",
    disabled=chat_feed.param.disabled,
)
undo_button = pn.widgets.Button(
    name="Undo",
    on_click=lambda event: chat_feed.undo(2), align="center", disabled=chat_feed.param.disabled
)
clear_button = pn.widgets.Button(name="Clear", on_click=lambda event: chat_feed.clear(), align="center", disabled=chat_feed.param.disabled)


pn.Column(
    chat_feed,
    pn.layout.Divider(),
    pn.Row("Click a button", andrew_button, marc_button, philipp_button, undo_button, clear_button),
)

Note how we use emojis and images (via urls) as avatars in the `ChatFeed`.

We chose to use an *async* callback in the example. We recommend using *async* callbacks and *async* functions from `openai` and other AI frameworks whenever possible for optimal performance.